In [3]:
import pandas as pd
import json

In [5]:
fns = ['/home/eamonn/Projects/corpuscule/ycac/ycac_all.corpus']

In [6]:
df = pd.DataFrame()

In [9]:
for fn in fns:
    df = df.append(pd.read_csv(fn,encoding='latin1'))

In [10]:
len(df)

12275807

In [11]:
def str_to_vec(s):
    l = eval(s)
    rep = ""
    for i in range(12):
        if i in l:
            rep += "1"
        else:
            rep += "0"
    return rep

In [12]:
df.head(5)

,offset,PCsInNormalForm,file,Composer
0,3.000,[7],Danse Andalouse Eb major.mid,Ascher
1,3.500,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher
2,3.625,[0],Danse Andalouse Eb major.mid,Ascher
3,3.750,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher
4,3.875,[0],Danse Andalouse Eb major.mid,Ascher


In [13]:
df['canonical'] = df.PCsInNormalForm.apply(str_to_vec)

In [14]:
df.head(5)

,offset,PCsInNormalForm,file,Composer,canonical
0,3.000,[7],Danse Andalouse Eb major.mid,Ascher,000000010000
1,3.500,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher,100100001000
2,3.625,[0],Danse Andalouse Eb major.mid,Ascher,100000000000
3,3.750,"[8, 0, 3]",Danse Andalouse Eb major.mid,Ascher,100100001000
4,3.875,[0],Danse Andalouse Eb major.mid,Ascher,100000000000


In [15]:
u = df.file.unique()
len(u)

8729

In [16]:
df.head(10).file[1]

'Danse Andalouse Eb major.mid'

In [13]:
df.to_csv('all_canonical.csv')

In [16]:
meta = pd.read_csv('/home/eamonn/Projects/corpuscule/ycac/csv/metadata/YCAC-metadata.csv')

CParserError: Error tokenizing data. C error: Buffer overflow caught - possible malformed input file.


In [43]:
meta.Date.fillna(meta.Range, inplace=True)

In [49]:
import numpy as np

def process_range(d):
    try:
        if '-' in d:
            a = int(d.split('-')[0])
            b = int(d.split('-')[1])
            return (a + b) / 2
        else:
            return int(d)
    except:
        return np.nan

In [50]:
meta['EstDate'] = meta.Date.apply(process_range)

In [53]:
df['file'] = df.file.apply(lambda s: s.replace('.mid', ''))
df['Filename'] = df['file']

In [54]:
master = pd.merge(df, meta, on='Filename')

In [55]:
master.to_csv('all_canonical.csv')

In [17]:
master = pd.read_csv('all_canonical.csv', dtype={'canonical':object})

## Dump to json

In [19]:
composers = list(master.Composer.value_counts().keys()[:20])

In [20]:
top_20_comp

['Mozart', 'Haydn', 'Bach', 'Handel', 'Casadesus']

In [1]:
# dump per composer


import json

for composer in top_20_comp:
    sub = master[master.Composer_x == composer]
    sub = sub[['file', 'canonical']]

    files = list(sub.file)
    vecs = list(sub.canonical)

    fv = pd.DataFrame(zip(files,vecs),columns = ['file', 'canonical'])
    fv.reset_index()
    
    done = fv.groupby('file')['canonical'].apply(list)
    docs = list(done)

    print("{} group done".format(composer))

    with open('{}.json'.format(composer), 'w') as f:
        json.dump(docs, f)

    print("{} dump done".format(composer))

NameError: name 'top_20_comp' is not defined

In [19]:
master = df

In [20]:
# dump whole corpus

sub = master
sub = sub[['file', 'canonical']]

files = list(sub.file)
vecs = list(sub.canonical)

fv = pd.DataFrame(list(zip(files,vecs)),columns = ['file', 'canonical'])
fv.reset_index()

done = fv.groupby('file')['canonical'].apply(list)
docs = list(done)

composer = 'ycac_all'

print("{} group done".format(composer))

with open('{}.json'.format(composer), 'w') as f:
    json.dump(docs, f)

print("{} dump done".format(composer))

ycac_all group done
ycac_all dump done


In [3]:
eras = [
    (1650, 1699),
    (1700, 1749),
    (1750, 1799),
    (1800, 1849),
    (1850, 1899),
    (1900, 1950)
]

In [ ]:
for era in eras:
    start_year = era[0]
    end_year = era[1]
    
    sub = master[master.EstDate.isin(range(start_year, end_year))]
    sub = sub[['file', 'canonical']]

    label = "{}_to_{}".format(start_year, end_year)
    
    files = list(sub.file)
    vecs = list(sub.canonical)

    fv = pd.DataFrame(zip(files,vecs),columns = ['file', 'canonical'])
    fv.reset_index()
    
    done = fv.groupby('file')['canonical'].apply(list)
    docs = list(done)

    print("{} group done".format(label))

    with open('{}.json'.format(label), 'w') as f:
        json.dump(docs, f)

    print("{} dump done".format(label))

1650_to_1699 group done
1650_to_1699 dump done
1700_to_1749 group done
1700_to_1749 dump done